<a href="https://colab.research.google.com/github/anapitalai/gee/blob/main/geemap_gee.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
import geemap
import ee
ee.Authenticate()
ee.Initialize(project="anapitalai-educational")

In [26]:
Map=geemap.Map()
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [14]:
butuwin_road_roi=Map.user_roi

In [20]:
# Load a Landsat 8 image collection and filter by date and location

collection = ee.ImageCollection('LANDSAT/LC08/C02/T1_RT') \
    .filterDate('2024-01-01', '2025-01-24') \
    .filterBounds(ee.Geometry.Point([152.22,-4.35]))  # Adjust the location as needed

# Define a function to calculate NDVI for each image in the collection
def calculate_ndvi(image):
    nir = image.select('B5')  # Select NIR band for Landsat 8
    red = image.select('B4')  # Select Red band for Landsat 8
    ndvi = nir.subtract(red).divide(nir.add(red)).rename('NDVI')
    return image.addBands(ndvi)

# Apply the function to the image collection
ndvi_collection = collection.map(calculate_ndvi)

# Get a specific image from the collection (e.g., the first image)
ndvi_image = ee.Image(ndvi_collection.first())

# Define visualization parameters
ndvi_params = {
    'min': -1,
    'max': 1,
    'palette': ['blue', 'white', 'green']
}


Map.centerObject(ndvi_image, 10)
Map.addLayer(ndvi_image.select('NDVI'), ndvi_params, 'NDVI')
Map.addLayerControl()  # This will add a layer control panel to the map


# Save the NDVI image to a file
#geemap.ee_export_image(ndvi_image.select('NDVI'), filename='ndvi_image.tif',)




In [19]:
# Load the Sentinel-2 image collection and filter by date and location
collection = ee.ImageCollection('COPERNICUS/S2') \
    .filterDate('2019-06-01', '2019-06-30') \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) \
    .filterBounds(ee.Geometry.Point([152.22,-4.35]))


# Define a function to calculate NDVI for each image in the collection
def calculate_ndvi(image):
    nir = image.select('B8')  # Select NIR band
    red = image.select('B4')  # Select Red band
    ndvi = nir.subtract(red).divide(nir.add(red)).rename('NDVI')
    return image.addBands(ndvi)

# Apply the function to the image collection
ndvi_collection = collection.map(calculate_ndvi)

# Get a specific image from the collection (e.g., the first image)
ndvi_image = ee.Image(ndvi_collection.first())

# Define visualization parameters for NDVI
ndvi_params = {
    'min': -1,
    'max': 1,
    'palette': ['blue', 'white', 'green']
}

# Create a map and display the NDVI layer
Map = geemap.Map()
Map.centerObject(ndvi_image, 10)
Map.addLayer(ndvi_image.select('NDVI'), ndvi_params, 'NDVI-Sentinel')
Map.addLayerControl()  # This will add a layer control panel to the map
Map
# Save the NDVI image to a file
#geemap.ee_export_image(ndvi_image.select('NDVI'), filename='ndvi_image.tif', scale=30)


Map(center=[-4.114990791418045, 152.59322858799445], controls=(WidgetControl(options=['position', 'transparent…

In [38]:
# Load Landsat 8 image collection and filter by date and location
# collection = ee.ImageCollection('LANDSAT/LC08/C02/T1_RT') \
#     .filterDate('2020-01-01', '2020-12-31') \
#     .filterBounds(ee.Geometry.Point([152.22,-4.35]))
#collection = ee.ImageCollection('LANDSAT/LC08/C02/T1_RT') \
collection=ee.ImageCollection("LANDSAT/LC08/C02/T1_L2") \
    .filterDate('2023-01-01', '2025-01-24') \
    .filterMetadata('CLOUD_COVER', 'less_than', 10) \
    .filterBounds(ee.Geometry.Point([152.22,-4.35]))

# Select a single image from the collection
image = collection.first()

# Apply cloud masking using the QA band
# def mask_clouds(image):
#     qa = image.select('QA_PIXEL')
#     cloud_mask = qa.bitwiseAnd(1 << 3).eq(0)  # Cloud bit
#     shadow_mask = qa.bitwiseAnd(1 << 4).eq(0)  # Cloud shadow bit
#     mask = cloud_mask.And(shadow_mask)
#     return image.updateMask(mask)

# masked_image = mask_clouds(image)

# Convert to Top-of-Atmosphere (TOA) reflectance
# toa_image = masked_image.divide(10000).select(['SRB2', 'B3', 'B4', 'B5', 'B6', 'B7'],
#                                               ['B2', 'B3', 'B4', 'B5', 'B6', 'B7'])

# Select relevant bands
red = image.select('SR_B4')
nir = image.select('SR_B5')
swir = image.select('SR_B6')

# Calculate NDRI
ndri = swir.subtract(nir).divide(swir.add(nir)).rename('NDRI')

# Define visualization parameters
ndri_params = {
    'min': -1,
    'max': 1,
    'palette': ['blue', 'white', 'green']
}

# Create a map and display the NDRI layer
Map = geemap.Map()
Map.centerObject(image, 10)
Map.addLayer(ndri, ndri_params, 'NDRI')
Map.addLayerControl()
Map


Map(center=[-4.341092935952704, 152.35766603182913], controls=(WidgetControl(options=['position', 'transparent…